In [1]:
import numpy as np
import pandas as pd
import pypsa  
import os 
import sys  
import datetime
from pyomo.environ import Constraint

file_name =  os.path.basename(sys.argv[0])
directory = os.path.dirname(os.path.realpath(file_name))+'/Output'
n_file = 'Output/' + [s for s in os.listdir(directory) if 'EU_Network_created' in s][0]

nz = pypsa.Network()
nz.import_from_netcdf('Output/nz_10hour.nc')
n2 = pypsa.Network()
n2.import_from_netcdf(n_file)

n_storage = pd.read_csv('Output/n_storage.csv',index_col=0)
n_storage.index = pd.to_datetime(n_storage.index)
Indices = np.load('Output/indices.npy')

INFO:pypsa.io:Imported network nz_8hour.nc has buses, carriers, generators, lines, links, loads, storage_units
INFO:pypsa.io:Imported network EU_Network_created_25102020.nc has buses, carriers, generators, lines, links, loads, storage_units


In [4]:
#initialize dictionary of buses per country
Country2Bus = {}
for i in nz.buses['country'].unique():
    Country2Bus[i] = nz.buses[nz.buses['country'] == i].index

#add country tags to generators and storage units (for convenience)
nz.generators['country'] = np.array(nz.buses.loc[nz.generators['bus'],'country'])
nz.storage_units['country'] = np.array(nz.buses.loc[nz.storage_units['bus'],'country'])

#Get the list of storage units with significant standard deviation over the course of the year
Storage_units = nz.storage_units.loc[n_storage.std(axis=0)>100000].index

In [5]:
#Rules used for extra_functionality - gathered here because they are used at different times for different cases

def balance_rule(model,c):
    Country_Gens = nz.generators[nz.generators['country'] == c]
    Country_Gens = Country_Gens[~Country_Gens.index.str.contains('EmGenUp2|EmGenDown')].index
    Country_Storage = nz.storage_units[nz.storage_units['country'] == c].index 
    Gen_p = sum([model.generator_p[i,nz.snapshots[hour]] for i in Country_Gens])
    Storage_store = sum([model.storage_p_store[i,nz.snapshots[hour]] for i in Country_Storage])
    Storage_disp = sum([model.storage_p_dispatch[i,nz.snapshots[hour]] for i in Country_Storage])
    zonal_gen = nz.generators_t.p.loc[nz.snapshots[hour],Country_Gens].sum()
    zonal_storage = nz.storage_units_t.p.loc[nz.snapshots[hour],Country_Storage].sum()
    return  Gen_p + Storage_disp - Storage_store >= zonal_gen + zonal_storage

def storage_rule(model,su):
    final = n_storage.loc[nz.snapshots[hour],su]
    start = n_storage.loc[nz.snapshots[hour-1],su]
    if (final-start > 0) and ('hydro' in su):
        inflow = nz.storage_units_t.inflow.loc[nz.snapshots[hour],su]
        if start + inflow > final:
            return model.state_of_charge[su,nz.snapshots[hour]] >= final
        else:
            return model.state_of_charge[su,nz.snapshots[hour]] >= start + inflow
    else:
        return model.state_of_charge[su,nz.snapshots[hour]] >= final
    
def extra_functionality(network,snapshots):
    model = network.model
    model.balance = Constraint(nz.buses['country'].unique(),rule=balance_rule)
    model.storage = Constraint(Storage_units,rule=storage_rule)

In [6]:
n2.storage_units['cyclic_state_of_charge'] = False

#enable emergency balancing gens
n2.generators.loc[n2.generators.index.str.contains('EmGenUp2'),'p_max_pu'] = 1
n2.generators.loc[n2.generators.index.str.contains('EmGenDown'),'p_min_pu'] = -1

# Run optimization for desired timesteps
for i in Indices: 

    print('Hour: ',i)
    print(datetime.datetime.now())
    hour = i
    timesteps = np.array([hour])
    
    n2.storage_units['state_of_charge_initial'] = n_storage.iloc[hour-1]
    n2.lopf(nz.snapshots[timesteps],solver_name='gurobi',extra_functionality=extra_functionality)

print(datetime.datetime.now())

n2.export_to_netcdf('Output/n2_10hour.nc')
n2.generators_t.p.iloc[Indices].to_csv('Output/n2_gen_10h.csv')
n2.storage_units_t.p.iloc[Indices].to_csv('Output/n2_storage_10h.csv')

Hour:  2723
2020-11-01 16:32:28.732478


INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 1135694763.792902
  Upper bound: 1135694763.792902
  Number of objectives: 1
  Number of constraints: 14520
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 81147
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination me

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 377000855.2530165
  Upper bound: 377000855.2530165
  Number of objectives: 1
  Number of constraints: 14520
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 81147
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination me

...

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 11962368640.723637
  Upper bound: 11962368640.723637
  Number of objectives: 1
  Number of constraints: 14520
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 81147
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination 

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 371810940.38467836
  Upper bound: 371810940.38467836
  Number of objectives: 1
  Number of constraints: 14520
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 81147
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination 

INFO:pypsa.io:Exported network n2_8hour.nc has storage_units, generators, lines, buses, carriers, links, loads


<xarray.Dataset>
Dimensions:                                (buses_i: 3037, buses_t_marginal_price_i: 3037, buses_t_p_i: 2407, buses_t_v_ang_i: 3030, carriers_i: 15, generators_i: 16013, generators_t_p_i: 6036, generators_t_p_max_pu_i: 5827, lines_i: 4428, lines_t_mu_lower_i: 182, lines_t_mu_upper_i: 195, lines_t_p0_i: 4424, lines_t_p1_i: 4424, links_i: 26, links_t_mu_lower_i: 20, links_t_mu_upper_i: 15, links_t_p0_i: 26, links_t_p1_i: 26, loads_i: 2388, loads_t_p_i: 2388, loads_t_p_set_i: 2388, snapshots: 8760, storage_units_i: 507, storage_units_t_inflow_i: 379, storage_units_t_p_i: 505, storage_units_t_spill_i: 90, storage_units_t_state_of_charge_i: 507)
Coordinates:
  * snapshots                              (snapshots) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
  * storage_units_i                        (storage_units_i) object '1261 PHS' ... '8017 PHS'
  * storage_units_t_inflow_i               (storage_units_t_inflow_i) object '1261 hydro' ... '7998 hydro'
  * storage_units_t_p_i                    (storage_units_t_p_i) object '1261 PHS' ... '8017 PHS'
  * storage_units_t_state_of_charge_i      (storage_units_t_state_of_charge_i) object '1261 PHS' ... '8017 PHS'
  * storage_units_t_spill_i                (storage_units_t_spill_i) object '1261 hydro' ... '7815 hydro'
  * generators_i                           (generators_i) object '1086 offwind-ac' ... 'EmGenDown8067'
  * generators_t_p_max_pu_i                (generators_t_p_max_pu_i) object '1086 offwind-ac' ... '999 solar'
  * generators_t_p_i                       (generators_t_p_i) object '1086 onwind' ... 'EmGenUp17964'
  * lines_i                                (lines_i) object '10236' ... '7308'
  * lines_t_p0_i                           (lines_t_p0_i) object '10236' ... '7308'
  * lines_t_p1_i                           (lines_t_p1_i) object '10236' ... '7308'
  * lines_t_mu_lower_i                     (lines_t_mu_lower_i) object '10705' ... '4452'
  * lines_t_mu_upper_i                     (lines_t_mu_upper_i) object '10133' ... '4360'
  * buses_i                                (buses_i) object '6964' ... '8067'
  * buses_t_p_i                            (buses_t_p_i) object '6974' ... '8067'
  * buses_t_v_ang_i                        (buses_t_v_ang_i) object '6964' ... '8067'
  * buses_t_marginal_price_i               (buses_t_marginal_price_i) object '6964' ... '8067'
  * carriers_i                             (carriers_i) object 'CCGT' ... 'ror'
  * links_i                                (links_i) object '4436' ... '11384+2'
  * links_t_p0_i                           (links_t_p0_i) object '4436' ... '11384+2'
  * links_t_p1_i                           (links_t_p1_i) object '4436' ... '11384+2'
  * links_t_mu_lower_i                     (links_t_mu_lower_i) object '4436' ... '11384+2'
  * links_t_mu_upper_i                     (links_t_mu_upper_i) object '4436' ... '499'
  * loads_i                                (loads_i) object '1086' ... '999'
  * loads_t_p_set_i                        (loads_t_p_set_i) object '1086' ... '999'
  * loads_t_p_i                            (loads_t_p_i) object '1086' ... '999'
Data variables:
    snapshots_weightings                   (snapshots) float64 1.0 1.0 ... 1.0
    storage_units_max_hours                (storage_units_i) float64 6.0 ... 6.0
    storage_units_p_min_pu                 (storage_units_i) float64 -1.0 ... -1.0
    storage_units_efficiency_dispatch      (storage_units_i) float64 0.866 ... 0.866
    storage_units_p_nom                    (storage_units_i) float64 384.0 ... 11.0
    storage_units_capital_cost             (storage_units_i) float64 1.606e+05 ... 1.606e+05
    storage_units_bus                      (storage_units_i) object '1261' ... '8017'
    storage_units_carrier                  (storage_units_i) object 'PHS' ... 'PHS'
    storage_units_efficiency_store         (storage_units_i) float64 0.866 ... 0.866
    storage_units_state_of_charge_initial  (storage_units_i) float64 864.0 ... 41.25